##Import library & Load the dataset from gdrive

In [0]:
# Import all the necessary library & files!
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
drive.mount('/content/gdrive')

## Initialization

In [0]:
image_path = './gdrive/My Drive/24hackton/images'
model_path = './gdrive/My Drive/24hackton/models'
train_img_path = '{}/train'.format(image_path)
val_img_path = '{}/val'.format(image_path)

## Data augmentation

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rotation_range=25,
        vertical_flip=True,
        fill_mode='reflect',
        data_format='channels_last',
        brightness_range=[0.5, 1.5],
        featurewise_center=True,
        featurewise_std_normalization=True)


val_datagen = ImageDataGenerator(
    rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_img_path,
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_img_path,
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical')

In [0]:
print (train_generator.class_indices)
Labels = '\n'.join(sorted(train_generator.class_indices.keys()))
print (Labels)